**THIS NOTEBOOK HAS BEEN REFERED FROM https://towardsdatascience.com/twitter-sentiment-analysis-nlp-text-analytics-b7b296d71fce THE DATA IS IN THIS MAIL ID'S GOOGLE DRIVE**

https://towardsdatascience.com/you-are-what-you-tweet-7e23fb84f4ed


In [1]:
import pandas as pd
import re
import nltk

**READ DATA**

In [3]:
train_data = pd.read_csv('/content/drive/My Drive/archive/train.csv')

**EDA - TO SEE ROWS AND COLUMNS, COUNT, NULL VALUES**

In [5]:
print(train_data.count())

id       31962
label    31962
tweet    31962
dtype: int64


In [6]:
train_data.isnull().any()

id       False
label    False
tweet    False
dtype: bool

In [8]:
train_data.tail()

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


**DATA CLEANING** 

In [9]:
import string
string.punctuation #type string

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def removePunc(text):
  no_punct = (words for words in text if words not in string.punctuation)
  #print(no_punct)
  no_punct_word = ''.join(no_punct)
  return no_punct_word

In [21]:
train_data['tweet_wo_punct'] = None
for i in range (train_data.shape[0]):
  output = removePunc(train_data['tweet'][i])
  train_data['tweet_wo_punct'][i] = output
  #print(output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [22]:
#train_data = train_data.drop(columns = ['tweet_wo_punct'])
train_data

,id,label,tweet,tweet_wo_punct
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is so...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i cant use ca...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in u...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate user isz that youuuðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",user sikh temple vandalised in in calgary wso ...


**TRAIN TEST SPLIT**

In [23]:
from sklearn.model_selection import train_test_split
y = train_data.label.values
x_train, x_test, y_train, y_test = train_test_split(train_data.tweet_wo_punct.values, y, stratify=y ,random_state=1, test_size=0.3, shuffle=True)

In [24]:
train_data.tweet_wo_punct.values

array([' user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction   run',
       'user user thanks for lyft credit i cant use cause they dont offer wheelchair vans in pdx    disapointed getthanked',
       '  bihday your majesty', ...,
       'listening to sad songs on a monday morning otw to work is sad  ',
       'user sikh temple vandalised in in calgary wso condemns  act  ',
       'thank you user for you follow  '], dtype=object)

**COUNTVECTORIZER**

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, stop_words='english')
vectorizer.fit((x_train))

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [26]:
print(vectorizer.get_feature_names())

['0000001', '00027', '001', '00h30', '01', '0115', '01926889917', '01b', '02', '0266808099', '02900', '03', '030916', '03111880779', '032', '0345', '039', '04', '045', '04k', '05', '0506823156', '06', '06052016', '060616', '0608wed', '0609', '0610', '061116', '0612', '0613', '0616', '0616ç', '0617', '0618â', '0620', '06202016', '0622', '0624', '06â', '07000', '07040', '07190', '07400', '07500', '076', '07800', '07840', '07850', '07870', '07950', '07b', '08', '0806', '080616', '08â', '0933m', '09600', '0k', '0shares', '0tolerancemovie', '10', '100', '1000', '100000', '10003', '1000gifts', '1000th', '1000â', '1001', '1001000s', '10014', '100616', '10064', '100daysofcode', '100daysofpigpaintings', '100daysoftea', '100faces', '100happydays', '100happysongs', '100k', '100ml', '100pm', '101', '10125', '1014', '10143hr', '1015', '1017', '101dalmatians', '101daysofsmiles', '101ð', '1027', '102816', '1030', '1044', '10480', '1059am', '105kg', '106', '10650', '10670', '1070', '10700', '1080', '1

In [27]:
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [28]:
print(x_test_vec.toarray().shape)

(9589, 34346)


**SVM LINEAR KERNEL**

In [29]:
from sklearn import svm
svm = svm.SVC(kernel = 'linear', probability=True)
prob = svm.fit(x_train_vec, y_train)
y_pred_svm = svm.predict(x_test_vec)

In [30]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  96.3708415893211 %


In [31]:
from sklearn.metrics import confusion_matrix
y_true = y_test
y_pred = y_pred_svm
confusion_matrix(y_true, y_pred)

array([[8813,  103],
       [ 245,  428]])

**SVM LINEAR SIGMOID**

In [38]:
from sklearn import svm
svm1 = svm.SVC(kernel = 'sigmoid', probability=True)
prob1 = svm1.fit(x_train_vec, y_train)
#.predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm_s = svm1.predict(x_test_vec)

In [39]:
print("Accuracy score for SVC_sigmoid is: ", accuracy_score(y_test, y_pred_svm_s) * 100, '%')

Accuracy score for SVC_sigmoid is:  95.89112524767962 %


In [40]:
y_true = y_test
y_pred = y_pred_svm_s
confusion_matrix(y_true, y_pred)

array([[8883,   33],
       [ 361,  312]])

**LOGISTIC REGRESSION**

In [32]:
from sklearn.linear_model import LogisticRegression
output_lr = LogisticRegression(random_state=0).fit(x_train_vec, y_train)
y_pred_lr = output_lr.predict(x_test_vec)

In [33]:
print("Accuracy score for LOGISTIC REGRESSION is: ", accuracy_score(y_test, y_pred_lr) * 100, '%')

Accuracy score for LOGISTIC REGRESSION is:  96.05798310564188 %


In [34]:
y_true = y_test
y_pred = y_pred_lr
confusion_matrix(y_true, y_pred)

array([[8866,   50],
       [ 328,  345]])